# 说在前面

这里需要调的“参数”有：

选择的特征，比如是否需要删除某些特征

划分验证集的方式

特征归一化的方式，比如z-score, min-max

In [71]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr
import pandas as pd
import os
from tqdm import tqdm, tnrange, tqdm_notebook

DF = pd.DataFrame
arr = np.array

# 划分特征以及特征编码

## 读取训练集和测试集

In [72]:
dirPath = "data preprocessed - step 3"
trainSet_origin = pd.read_csv(dirPath + '\\train.csv')
testSet_origin = pd.read_csv(dirPath + '\\test.csv')

trainSet_origin.shape, testSet_origin.shape

((16615, 13), (742, 13))

In [73]:
trainSet_origin.head()

,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,season,holiday,weekday,workingday,cnt
0,1,2011/1/1,0,1,9.84,14.395,81,0.0,1,0,6,0,16
1,2,2011/1/1,1,1,9.02,13.635,80,0.0,1,0,6,0,40
2,3,2011/1/1,2,1,9.02,13.635,80,0.0,1,0,6,0,32
3,4,2011/1/1,3,1,9.84,14.395,75,0.0,1,0,6,0,13
4,5,2011/1/1,4,1,9.84,14.395,75,0.0,1,0,6,0,1


In [74]:
testSet_origin.head()

,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,season,holiday,weekday,workingday,cnt
0,16638,2012/12/1,0,1,10.66,15.150,81,0.0000,4,0,6,0,?
1,16639,2012/12/1,1,1,10.66,15.150,81,0.0000,4,0,6,0,?
2,16640,2012/12/1,2,2,10.66,15.150,81,0.0000,4,0,6,0,?
3,16641,2012/12/1,3,2,10.66,13.635,81,8.9981,4,0,6,0,?
4,16642,2012/12/1,4,1,10.66,14.395,81,6.0032,4,0,6,0,?


## 去除无关特征

In [75]:
useless_features = ['instant']
trainSet_origin.drop(useless_features, axis=1, inplace=True)
testSet_origin.drop(useless_features, axis=1, inplace=True)

In [76]:
trainSet_origin.head()
testSet_origin.head()

,dteday,hr,weathersit,temp,atemp,hum,windspeed,season,holiday,weekday,workingday,cnt
0,2011/1/1,0,1,9.84,14.395,81,0.0,1,0,6,0,16
1,2011/1/1,1,1,9.02,13.635,80,0.0,1,0,6,0,40
2,2011/1/1,2,1,9.02,13.635,80,0.0,1,0,6,0,32
3,2011/1/1,3,1,9.84,14.395,75,0.0,1,0,6,0,13
4,2011/1/1,4,1,9.84,14.395,75,0.0,1,0,6,0,1


,dteday,hr,weathersit,temp,atemp,hum,windspeed,season,holiday,weekday,workingday,cnt
0,2012/12/1,0,1,10.66,15.150,81,0.0000,4,0,6,0,?
1,2012/12/1,1,1,10.66,15.150,81,0.0000,4,0,6,0,?
2,2012/12/1,2,2,10.66,15.150,81,0.0000,4,0,6,0,?
3,2012/12/1,3,2,10.66,13.635,81,8.9981,4,0,6,0,?
4,2012/12/1,4,1,10.66,14.395,81,6.0032,4,0,6,0,?


## 划分dteday特征

dteday特征包含年月日信息，可能对算法有用，因此打算将该特征划分为三个特征。

首先看一下训练集和测试集在dteday这个特征的数据分布。

In [77]:
def test_dteday(df):
    date = list(df.dteday.str.split("/"))

    year = [int(i[0]) for i in date]
    month = [int(i[1]) for i in date]
    day = [int(i[2]) for i in date]

    print(list(Counter(year).keys()))
    print(list(Counter(month).keys()))
    print(list(Counter(day).keys()))
    
    
test_dteday(trainSet_origin)
'---------'
test_dteday(testSet_origin)

[2011, 2012]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


'---------'

[2012]
[12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


从这里也可以看到，测试中数据只包含年份2012和月份12的数据。

但是月份这个特征还待考量。虽然测试集上只有12月的数据，但是在使用验证集测试参数时可能会使用到月份数据，因此还是保留月份。

至于日这个特征，暂时先保留。可能有用，后续再测试要不要去除。（TODO:测试下这个特征对数据的影响）

In [78]:
date = list(trainSet_origin.dteday.str.split("/"))

year = [int(i[0]) for i in date]
month = [int(i[1]) for i in date]
day = [int(i[2]) for i in date]

trainSet_origin.insert(0, 'year', year)
trainSet_origin.insert(1, 'month', month)
#trainSet_origin.insert(2, 'day', day)
trainSet_origin.drop(['dteday'], axis=1, inplace=True)

trainSet_origin.shape
trainSet_origin.head()

(16615, 13)

,year,month,hr,weathersit,temp,atemp,hum,windspeed,season,holiday,weekday,workingday,cnt
0,2011,1,0,1,9.84,14.395,81,0.0,1,0,6,0,16
1,2011,1,1,1,9.02,13.635,80,0.0,1,0,6,0,40
2,2011,1,2,1,9.02,13.635,80,0.0,1,0,6,0,32
3,2011,1,3,1,9.84,14.395,75,0.0,1,0,6,0,13
4,2011,1,4,1,9.84,14.395,75,0.0,1,0,6,0,1


In [79]:
date = list(testSet_origin.dteday.str.split("/"))

year = [int(i[0]) for i in date]
month = [int(i[1]) for i in date]
day = [int(i[2]) for i in date]

testSet_origin.insert(0, 'year', year)
testSet_origin.insert(1, 'month', month)
#testSet_origin.insert(2, 'day', day)
testSet_origin.drop(['dteday'], axis=1, inplace=True)

testSet_origin.shape
testSet_origin.head()

(742, 13)

,year,month,hr,weathersit,temp,atemp,hum,windspeed,season,holiday,weekday,workingday,cnt
0,2012,12,0,1,10.66,15.150,81,0.0000,4,0,6,0,?
1,2012,12,1,1,10.66,15.150,81,0.0000,4,0,6,0,?
2,2012,12,2,2,10.66,15.150,81,0.0000,4,0,6,0,?
3,2012,12,3,2,10.66,13.635,81,8.9981,4,0,6,0,?
4,2012,12,4,1,10.66,14.395,81,6.0032,4,0,6,0,?


## 合并训练集和测试集

由于后续需要对训练集和测试集都进行同样的数据处理，在处理过程中可能会涉及到数据集维度的变化，因此这里事先把两个数据集合并起来，以免后续因为维度不统一而出现问题。

In [80]:
allDataSet_origin = pd.concat([trainSet_origin, testSet_origin], axis=0)
allDataSet_origin.shape
allDataSet_origin.head()

(17357, 13)

,year,month,hr,weathersit,temp,atemp,hum,windspeed,season,holiday,weekday,workingday,cnt
0,2011,1,0,1,9.84,14.395,81,0.0,1,0,6,0,16
1,2011,1,1,1,9.02,13.635,80,0.0,1,0,6,0,40
2,2011,1,2,1,9.02,13.635,80,0.0,1,0,6,0,32
3,2011,1,3,1,9.84,14.395,75,0.0,1,0,6,0,13
4,2011,1,4,1,9.84,14.395,75,0.0,1,0,6,0,1


In [81]:
allDataSet_origin.columns

Index(['year', 'month', 'hr', 'weathersit', 'temp', 'atemp', 'hum',
       'windspeed', 'season', 'holiday', 'weekday', 'workingday', 'cnt'],
      dtype='object')

In [82]:
for i in ['weathersit','season','holiday','weekday']:
    allDataSet_origin[i] = allDataSet_origin[i].astype("int64")
    print(Counter(allDataSet_origin[i]))

Counter({1: 11399, 2: 4536, 3: 1419, 4: 3})
Counter({3: 4489, 2: 4404, 1: 4235, 4: 4229})
Counter({0: 16858, 1: 499})
Counter({6: 2509, 0: 2501, 5: 2482, 1: 2475, 3: 2472, 4: 2468, 2: 2450})


## 对离散变量进行编码

In [83]:
categorical_features = ['year', 'month', 'hr', 'weathersit','season','holiday','weekday'] #所有离散变量 # 去掉了 , 'day'

def encodeCategoricalFeatures(dataSet):
    for feature in categorical_features:
        #创键 哑变量
        dummies = pd.get_dummies(dataSet[feature], prefix=feature, drop_first=False)
        #合并数据集
        dataSet = pd.concat([dummies, dataSet], axis=1)
    return dataSet.drop(categorical_features, axis=1)

allDataSet = encodeCategoricalFeatures(allDataSet_origin)

allDataSet.shape
allDataSet.columns
allDataSet.head()

(17357, 61)

Index(['weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'holiday_0', 'holiday_1', 'season_1',
       'season_2', 'season_3', 'season_4', 'weathersit_1', 'weathersit_2',
       'weathersit_3', 'weathersit_4', 'hr_0', 'hr_1', 'hr_2', 'hr_3', 'hr_4',
       'hr_5', 'hr_6', 'hr_7', 'hr_8', 'hr_9', 'hr_10', 'hr_11', 'hr_12',
       'hr_13', 'hr_14', 'hr_15', 'hr_16', 'hr_17', 'hr_18', 'hr_19', 'hr_20',
       'hr_21', 'hr_22', 'hr_23', 'month_1', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10',
       'month_11', 'month_12', 'year_2011', 'year_2012', 'temp', 'atemp',
       'hum', 'windspeed', 'workingday', 'cnt'],
      dtype='object')

,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,holiday_0,holiday_1,season_1,...,month_11,month_12,year_2011,year_2012,temp,atemp,hum,windspeed,workingday,cnt
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.84,14.395,81,0.0,0,16
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.02,13.635,80,0.0,0,40
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.02,13.635,80,0.0,0,32
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.84,14.395,75,0.0,0,13
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.84,14.395,75,0.0,0,1


## 还原训练集和测试集
对两个数据集要一并处理的步骤已经完成，这里先把两个数据集分开回去。

In [84]:
divideLine = testSet_origin.shape[0]
trainSet_origin2 = allDataSet[:-divideLine].astype('float64')
testSet_origin2 = allDataSet[-divideLine:].astype('float64', raise_on_error=False)

trainSet_origin2.shape, testSet_origin2.shape

((16615, 61), (742, 61))

In [85]:
assert(trainSet_origin2.shape[0] == trainSet_origin.shape[0])
assert(testSet_origin2.shape[0] == testSet_origin.shape[0])

trainSet_origin2.head()
testSet_origin2.head()

,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,holiday_0,holiday_1,season_1,...,month_11,month_12,year_2011,year_2012,temp,atemp,hum,windspeed,workingday,cnt
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.84,14.395,81.0,0.0,0.0,16.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.02,13.635,80.0,0.0,0.0,40.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.02,13.635,80.0,0.0,0.0,32.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.84,14.395,75.0,0.0,0.0,13.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.84,14.395,75.0,0.0,0.0,1.0


,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,holiday_0,holiday_1,season_1,...,month_11,month_12,year_2011,year_2012,temp,atemp,hum,windspeed,workingday,cnt
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,10.66,15.150,81.0,0.0000,0.0,?
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,10.66,15.150,81.0,0.0000,0.0,?
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,10.66,15.150,81.0,0.0000,0.0,?
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,10.66,13.635,81.0,8.9981,0.0,?
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,10.66,14.395,81.0,6.0032,0.0,?


## 去除测试集的无关数据

测试集最后一列（也即是cnt特征那一列）全部都是问号，这里便将其最后一列删除，以适应后续算法的数据要求。

In [86]:
testSet_origin2.drop(['cnt'], axis=1, inplace=True)
testSet_origin2.shape

(742, 60)

# 从训练集中划分验证集

典型的从训练集中划分验证集的方法是：**划分训练集中的splitRate%为验证集，划分过程采用随机选取的方式【第一种划分方式】**。但是这里**由于数据是有时间序列特性的，因此便采取划分训练集的后splitRate%为验证集【第二种划分方式】**。

但是又有一个问题，神经网络算法确定会利用时间序列特征吗？，不像RNN，其是没有记忆性的。

这里先测试第二种划分方式。

In [87]:
#划分比例
splitRate = 0.18
#划分的数目
splitNum = int(trainSet_origin2.shape[0]*splitRate) 
#得到 训练集 和验证集
trainSet = trainSet_origin2[:-splitNum].reset_index(drop=True)
validateSet = trainSet_origin2[-splitNum:].reset_index(drop=True)

trainSet.shape, validateSet.shape
trainSet.head(3)
#validateSet.head(3)

((13625, 61), (2990, 61))

,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,holiday_0,holiday_1,season_1,...,month_11,month_12,year_2011,year_2012,temp,atemp,hum,windspeed,workingday,cnt
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.84,14.395,81.0,0.0,0.0,16.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.02,13.635,80.0,0.0,0.0,40.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,9.02,13.635,80.0,0.0,0.0,32.0


# 归一化与不归一化

## 不归一化

### 保存未归一化的数据

In [88]:
testSet = testSet_origin2

In [89]:
trainSet.shape, validateSet.shape, testSet.shape

dirPath = "data preprocessed\\keep-all-necessary-feature\\unnormalized"
if not os.path.exists(dirPath):
    os.makedirs(dirPath)
    
trainSet.to_csv(dirPath + '\\train.csv', index=False, header=False)
validateSet.to_csv(dirPath + '\\validate.csv', index=False, header=False)
testSet.to_csv(dirPath + '\\test.csv', index=False, header=False)

((13625, 61), (2990, 61), (742, 60))

### 测试数据读取

In [90]:
t = np.loadtxt(dirPath + '\\test.csv', delimiter=",")
t.shape, t.dtype
t[:1]

((742, 60), dtype('float64'))

array([[  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,   0.  ,   0.  ,
          0.  ,   1.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   0.  ,   1.  ,  10.66,
         15.15,  81.  ,   0.  ,   0.  ]])

## 归一化

标准化连续特征

In [91]:
def normalizeFeature(train, validate, test, continousFeature):
    means_, stds_ = [], []
    for feature in continousFeature:
        mean_, std_ = train[feature].mean(), train[feature].std()
        featureIndex = list(trainSet.columns).index(feature)
        means_.append({featureIndex:mean_})
        stds_.append({featureIndex:std_})
        
        train[feature] = (train[feature] - mean_)/std_
        validate[feature] = (validate[feature] - mean_)/std_
        test[feature] = ((test[feature] - mean_)/std_).astype("float64")

    return train, validate, test, means_, stds_
        
#################### 在这里指定连续特征 ##############################    
continousFeatures = ["temp", "hum", "windspeed", 'atemp']
continousFeatures
#################### 在这里指定连续特征 ############################## 

trainSet, validateSet, testSet, means, stds = normalizeFeature(trainSet, validateSet, testSet, continousFeatures)

trainSet[continousFeatures].head(3)
validateSet[continousFeatures].head(3)
testSet[continousFeatures].head(3)

['temp', 'hum', 'windspeed', 'atemp']

,temp,hum,windspeed,atemp
0,-1.296485,0.94185,-1.57558,-1.060552
1,-1.398401,0.89161,-1.57558,-1.147282
2,-1.398401,0.89161,-1.57558,-1.147282


,temp,hum,windspeed,atemp
0,1.760997,-0.665811,1.327879,1.792408
1,1.760997,-0.766290,0.723194,1.705678
2,1.862913,-0.766290,0.843969,1.878567


,temp,hum,windspeed,atemp
0,-1.194568,0.94185,-1.57558,-0.974393
1,-1.194568,0.94185,-1.57558,-0.974393
2,-1.194568,0.94185,-1.57558,-0.974393


In [92]:
means,stds

([{55: 20.27130275229354},
  {57: 62.25277064220184},
  {58: 13.023422627523388},
  {56: 23.688439633027492}],
 [{55: 8.045836616515906},
  {57: 19.9046896960437},
  {58: 8.265797922320854},
  {56: 8.762828750866511}])

In [93]:
trainSet[continousFeatures].describe()
validateSet[continousFeatures].describe()
testSet[continousFeatures].describe()

,temp,hum,windspeed,atemp
count,1.362500e+04,1.362500e+04,1.362500e+04,1.362500e+04
mean,-1.567790e-15,-3.824871e-16,-5.322604e-14,2.226834e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.417561e+00,-3.127543e+00,-1.575580e+00,-2.703287e+00
25%,-7.869042e-01,-8.165297e-01,-7.285349e-01,-8.015037e-01
50%,2.842430e-02,-1.269905e-02,-3.075641e-03,6.294319e-02
75%,8.437528e-01,8.413710e-01,7.231942e-01,8.412307e-01
max,2.576326e+00,1.896399e+00,5.319931e+00,3.002633e+00


,temp,hum,windspeed,atemp
count,2990.000000,2990.000000,2990.000000,2990.000000
mean,0.292622,0.063030,-0.183782,0.280813
std,0.842672,0.823191,0.917900,0.836878
min,-1.500317,-2.323712,-1.575580,-1.579221
25%,-0.379240,-0.615572,-0.728535,-0.368995
50%,0.436089,0.138019,-0.244625,0.322563
75%,0.945669,0.740892,0.480834,0.927961
max,2.066746,1.896399,3.626653,2.138186


,temp,hum,windspeed,atemp
count,742.000000,742.000000,742.000000,742.000000
mean,-0.873299,0.302211,-0.073226,-0.862759
std,0.429266,0.891361,1.065969,0.486872
min,-1.806065,-1.570121,-1.575580,-1.838840
25%,-1.194568,-0.464854,-0.728535,-1.233442
50%,-0.888820,0.238498,-0.244625,-0.887663
75%,-0.583072,1.243286,0.480834,-0.455725
max,0.538005,1.896399,3.747427,0.841231


### 保存归一化的数据

In [94]:
trainSet.shape, validateSet.shape, testSet.shape

dirPath = "data preprocessed\\keep-all-necessary-feature\\normalized-[z-score]"
if not os.path.exists(dirPath):
    os.makedirs(dirPath)
    
trainSet.to_csv(dirPath + '\\train.csv', index=False, header=False)
validateSet.to_csv(dirPath + '\\validate.csv', index=False, header=False)
testSet.to_csv(dirPath + '\\test.csv', index=False, header=False)

((13625, 61), (2990, 61), (742, 60))

### 测试数据读取

In [95]:
t = np.loadtxt(dirPath + '\\test.csv', delimiter=",")
t.shape, t.dtype
t[:1]

((742, 60), dtype('float64'))

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
        -1.19456847, -0.97439307,  0.94184987, -1.57557961,  0.        ]])